In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex
!pip install transformers

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 944 (delta 5), reused 21 (delta 4), pack-reused 919
Receiving objects: 100% (944/944), 96.61 MiB | 26.59 MiB/s, done.
Resolving deltas: 100% (595/595), done.
     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 49.7MB/s 
     |████████████████████████████████| 860kB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=3982a6efd78b969034c46256dfb428e035211d3c89ffe83b0cef222f3e940431
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import torch

from bertviz import head_view, model_view
from transformers import BertTokenizer, BertModel, BertConfig, BertForTokenClassification

from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [0]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [0]:
from keras.preprocessing.sequence import pad_sequences

def read_in_val_data(filepath):
  print("reading ",filepath)
  genes = []
  labels = []
  df = pd.read_csv(filepath,usecols=[2,3],sep="\t",skiprows=1,header=None,nrows=10)
  print(df.head())
  for entry in df.itertuples():
    kmer_list = [kmer.strip("\'") for kmer in entry[1][1:-1].split(", ")]
    #for some reason there are empty examples in here
    if(len(kmer_list)>1):
      label_list = list(map(float, entry[2][1:-1].split(", ")))
      genes.append(kmer_list)
      labels.append(label_list)
  return genes, labels

def tokenize_and_pad_single_sample(sample,label):
  k= len(genes[0][0])
  if k==4:
    kmer_filepath = '/content/drive/My Drive/kmers/fourmers.txt'
  elif k==6:
    kmer_filepath = '/content/drive/My Drive/kmers/hexamers.txt'
  elif k==8:
    kmer_filepath = '/content/drive/My Drive/kmers/octamers.txt'

  formatted_sample = ['[CLS]']  + sample + ['[SEP]']
  formatted_label = [0] + label + [0]
  tokenizer = BertTokenizer(kmer_filepath, max_len=MAX_LEN)
  attention_mask = np.concatenate([np.ones(len(label)), np.zeros(MAX_LEN - len(label))])
  seq_ids =[tokenizer.convert_tokens_to_ids(formatted_sample)]
  sample_ids = pad_sequences(seq_ids, maxlen=MAX_LEN, truncating='post', padding='post')[0]
  
  return sample_ids, attention_mask, formatted_label

In [0]:
#FOR MSE
class BertForTokenRegression(torch.nn.Module):
  def __init__(self):
    super(BertForTokenRegression, self).__init__()
    self.config = BertConfig(vocab_size_or_config_json_file='/content/drive/My Drive/regression_bert_configuration.json')
    self.bert = BertForTokenClassification(self.config)
    self.linear = torch.nn.Linear(self.config.hidden_size, 1)
  def forward(self, input_ids, attention_mask=None):
    scores, hidden_states, att_weights = self.bert(input_ids,attention_mask=attention_mask)
    preds = self.linear(hidden_states[8]).squeeze(0).squeeze(1)
    return preds, att_weights

In [7]:
kmer_filepath='/content/drive/My Drive/kmers/hexamers.txt'
MAX_LEN=1002
model = BertForTokenRegression()
model.load_state_dict(torch.load("/content/drive/My Drive/attentive_splice_weights/brain_bert_weights.pt",map_location=torch.device('cpu')))

genes, labels = read_in_val_data('/content/drive/My Drive/partitioned_samples/tissue-specific/Brain_train.txt')

reading  /content/drive/My Drive/partitioned_samples/tissue-specific/Brain_train.txt
                                                   2                                                  3
0  ['CACCTG', 'TGGTCC', 'CAGCTA', 'TTCAGG', 'AAGC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1  ['GAATAA', 'TATTTT', 'TCTTTT', 'CTTTTT', 'TCCT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2  ['CAAGTA', 'TGAGAC', 'TGTGGA', 'CCATGC', 'TGAT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3  ['CGCCAT', 'GTTGGT', 'CAGGCT', 'TGTCTT', 'GACC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4  ['ATCCGG', 'CCCGTG', 'GCGACA', 'CAGAGC', 'AGGC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...


In [0]:
input_ids, mask, label = tokenize_and_pad_single_sample(genes[0],labels[0])

input_tensor = torch.tensor(input_ids).unsqueeze(0).long()
mask_tensor = torch.tensor(mask).unsqueeze(0).long()
preds, attention = model(input_tensor, attention_mask=mask_tensor)
#input_id_list = input_ids[0].tolist() # Batch index 0
#tokens = tokenizer.convert_ids_to_tokens(input_id_list)
print(len(attention))
print(attention[0].size())
call_html()

#raw_tokens=pad_sequences([genes[0]], maxlen=MAX_LEN, truncating='post', padding='post')[0]
raw_tokens=['[CLS]']  + genes[0] + ['[SEP]']
print(raw_tokens)

model_view(attention, raw_tokens)

8
torch.Size([1, 8, 1002, 1002])


['[CLS]', 'CACCTG', 'TGGTCC', 'CAGCTA', 'TTCAGG', 'AAGCTG', 'GGTTGG', 'GAGGAT', 'CACTTG', 'AGCCTG', 'GGAAGT', 'GAAGGC', 'TACAGT', 'GTGCTC', 'TGATTG', 'TGTTGC', 'TGCACT', 'CCAGTC', 'TGGGTG', 'ACAGAG', 'TGAGAC', 'CTCATC', 'TTAAAA', 'TAAAAG', 'GCTGGA', 'AAATAA', 'TGATTA', 'TTGTCA', 'GTTAAT', 'CAGAAA', 'CTGAAA', 'GGATCC', 'CCTGGA', 'ACAGCA', 'GCATCA', 'AGATAA', 'ATGTTT', 'GTTGAA', 'ACTGCA', 'GGTCGC', 'TTGCCC', 'TGTCTC', 'AGTGTG', 'ATGGGG', 'ATGGTG', 'TAAGGG', 'GTTTCT', 'GTGCTC', 'CCTGTG', 'GCAGAT', 'GGAGGC', 'ACCAAC', 'GGTAAT', 'GTTATT', 'TAACTT', 'TTGAAG', 'CTGTGT', 'CGAATA', 'TAAACA', 'ATCTCT', 'AAAAAT', 'CAACAC', 'ATGATT', 'TGGACA', 'TTATTG', 'ACTGAT', 'TCACTG', 'GGCCAC', 'TGGTGC', 'CAGCAC', 'AAGTCA', 'CCTGGT', 'AAACGC', 'TTGATA', 'GCCTAC', 'AACAAA', 'TGGCGT', 'TCACAA', 'GAATAA', 'ATGAGC', 'ATTTTC', 'TTTTCA', 'ACAACA', 'CCGTGG', 'GCCAGT', 'GTCTAT', 'GGCTTT', 'ACCACA', 'TGGCAT', 'CATTAT', 'TCACTG', 'TTTAGG', 'GTTGGT', 'CTTAAG', 'TATTTG', 'GTTTTA', 'GGGGTT', 'GTGATA', 'TGTTCA', 'CTTCAG', 